In [2]:
import os
os.chdir("..")
from infopath.config import load_training_opt
import copy
import json

In [3]:
# parameter_recovery : GoNoGo_nofb, GoNoGo_withfb1, GoNoGo_withfb2 # fb2 is the differential

log_path = f"log_dir/1d74764c4551eef5158418ea67fbe1a5885dfdb1/2024_5_27_9_46_33_teacher_conf"
opt = load_training_opt(log_path)
for dataset in ["GoNoGo_nofb_seed2", "GoNoGo_withfb1_seed2"]: #, "GoNoGo_withfb2_seed2"]:
    name = dataset.split("_")[-2]
    for seed in [0,1, 2,3,4]:
        for l1 in [0.02]:
            opt_student = copy.deepcopy(opt)
            opt_student.batch_size = 400
            opt_student.lr = 0.001
            opt_student.datapath = f"datasets/{dataset}" 
            opt_student.trial_onset = -0.052
            opt_student.block_graph = [] 
            opt_student.l1_decay_across = l1
            opt_student.l1_decay = 0.001
            opt_student.seed = seed
            opt_student.early_stop = 4000
            opt_student.geometric_loss = False
            opt_student.feat_svd = False
            
            opt_student.stats_loss = False
            opt_student.loss_cross_corr = False
            opt_student.coeff_cross_corr_loss = 1
            opt_student.spike_function = "sigmoid"

            # opt_student.loss_trial_wise = 0
            opt_student.restrict_inter_area_inh = True
            opt_student.flag_ei = True
            string = "_nospike"
            opt_student.hard_stop = 20000
            opt_student = vars(opt_student)
            opt_student["device"] = "cuda"
            opt_student["scale_fun"] = "log"
            opt_student.pop("log_path")
            opt_student.pop("wandb_run_id")
            if not os.path.exists(f"configs/grid_{name}/l1across{int(l1*10000)}_seed{seed}{string}"):
                os.makedirs(f"configs/grid_{name}/l1across{int(l1*10000)}_seed{seed}{string}")
            json.dump(opt_student, open(f"configs/grid_{name}/l1across{int(l1*10000)}_seed{seed}{string}/opt.json", "w"))
    # opt_student["loss_trial_wise"] = 0
    # if not os.path.exists(f"configs/grid_{name}/l1across{int(l1*10000)}_notm"):
    #     os.makedirs(f"configs/grid_{name}/l1across{int(l1*10000)}_notm")
    # json.dump(opt_student, open(f"configs/grid_{name}/l1across{int(l1*10000)}_notm/opt.json", "w"))


In [4]:
from infopath.config import compare_opt

In [9]:
compare_opt("configs/grid_nofb/l1across0_seed2", "log_dir/39a37b302593931ad8f1977c82e16327590f2805/2024_6_24_11_38_12_l1across0_seed0")

input_seed 2 0
seed 2 0
train_bias True False
train_noise_bias True False
train_input_weights True False


In [6]:
import json
# copy an existing for sparsity 
log_paths = [
    "log_dir/39a37b302593931ad8f1977c82e16327590f2805/2024_6_24_11_38_12_l1across0_seed0",
    "log_dir/39a37b302593931ad8f1977c82e16327590f2805/2024_6_24_11_38_16_l1across0_seed1",
    "log_dir/39a37b302593931ad8f1977c82e16327590f2805/2024_6_29_16_10_30_l1across0_seed2",
    "log_dir/39a37b302593931ad8f1977c82e16327590f2805/2024_6_29_16_10_30_l1across0_seed3",
    "log_dir/39a37b302593931ad8f1977c82e16327590f2805/2024_6_29_16_10_31_l1across0_seed4",        
    ]
for log_path in log_paths:
    !cp -r {log_path} {log_path}_prune 
    opt = json.load(open(f"{log_path}/opt.json"))
    opt["log_path"] += "_prune"
    opt["l1_decay"] = 0
    opt["iterative_pruning"] = True
    opt["log_every_n_steps"] = 200
    opt["early_stop"] = 8000
    json.dump(opt, open(f"{log_path}_prune/opt.json", "w"))
    


In [15]:
from infopath.config import compare_opt

compare_opt("log_dir/37383bc21e52b1a427bf2086b0659664c2fc1ccc/2024_10_22_12_40_45_l1across200_seed0_nospike", "log_dir/37383bc21e52b1a427bf2086b0659664c2fc1ccc/2024_9_23_16_7_26_l1across200_seed1")

datapath datasets/GoNoGo_nofb_seed2 datasets/DataFromVahid_expert
batch_size 400 200
lr 0.001 0.0005
l1_decay 0.001 0.0
n_units 500 1500
start -0.048 -0.1
stop 0.152 1.2
trial_onset -0.052 1.0
dt 1 2
n_rnn_in 1 2
input_seed 0 1
noise_level_list [0.14, 0.14] [0.14, 0.14, 0.14, 0.14, 0.14, 0.14]
tau_list [10, 10] [10, 10, 10, 10, 10, 10]
areas ['area1', 'area2'] ['wS1', 'wS2', 'wM1', 'wM2', 'ALM', 'tjM1']
stim [4] [0, 1]
spike_filter_std 8 12
reaction_time_limits None [-1, 0.2]
seed 0 1
coeff_trial_loss 10 1
restrict_inter_area_inh False True
train_noise_bias True False
early_stop 4000 8000
geometric_loss False True
motor_areas [] [0, 1, 2, 3, 4, 5]
jaw_delay 40 16
tau_jaw 50 5
trial_types [0, 3] [0, 1, 2, 3]
with_behaviour False True
jaw_open_loop False True
scaling_jaw_in_model False True
t_trial_gan True False
stim_onsets [0] [0, 1]
clip_grad 500 2000
time Thu Feb  8 17:59:52 2024 Mon Sep 23 16:07:26 2024
num_areas 2 6
flag_ei False True
train_input_weights False True
weights_random_d